# Notebook: Create OPA2VEC associations file
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Get genes associated with diseases (disgenet) and chemicals (ctdbase), take their go functions and assign to the associated chem/dis in an associations file. Also output finalclasses.lst, a file that tells opa2vec which entities you would like the vectors for

In [1]:
import pandas as pd
import numpy as np
import subprocess

### 1. Get Uniprot ID set for each Gene 

In [2]:
# CHEMICALS
# Import CTD file to get the geneIDs we want 
df_cg = pd.read_csv('../ctd-to-nt/csvs/CTD_chem_gene_ixns.csv', skiprows=27)
df_cg = df_cg.drop(0)

# Cut down to just humans if you want to
df_cg = df_cg.loc[df_cg['OrganismID'] == 9606.0]

# Need to change float to int for the url to work
df_cg['GeneID'] = df_cg.GeneID.astype(int)

In [3]:
# # DISEASES
# # Import DisGeNet with disease IDs and Gene IDs
# df_cgd = pd.read_csv('../disgenet-nt/input_tsvs/curated_gene_disease_associations.tsv', sep='\t')

# # Limit by evidence Score 
# df_cgd = df_cgd[df_cgd.score >= 0.42]

In [4]:
# df_cgd.head()

In [5]:
# Alternative DISEASES --> Switching out DisGeNet for CTD (as they use UMLS/MESH respectively)
# Import preprepared g-d associations (from ctd-to-nt notebook) direct assocs only
df_cgd = pd.read_csv('../ctd-to-nt/gene-dis-pos-assocs.csv')
df_cgd['GeneID'] = df_cgd['GeneID'].astype(int)
df_cgd.head()

,GeneID,DiseaseID
0,50518,MESH:D003920
1,50518,MESH:D003924
2,50518,MESH:D008113
3,50518,MESH:D009369
4,50518,MESH:D009765


In [6]:
# # Create list of all associated GeneIDs for each of Chemicals and Diseases
cgene_ids = df_cg.GeneID.unique()
dgene_ids = df_cgd.GeneID.unique()

# Combine them
all_genes = set(cgene_ids).union(set(dgene_ids))

In [7]:
df_cgd.head()

,GeneID,DiseaseID
0,50518,MESH:D003920
1,50518,MESH:D003924
2,50518,MESH:D008113
3,50518,MESH:D009369
4,50518,MESH:D009765


In [8]:
# Write combined gene ids to file
with open('geneIDs.txt', 'w') as f:
    for item in all_genes:
        f.write("%s\n" % item)

In [9]:
# NOTE the next step is MANUAL
# You need to go to https://www.uniprot.org/uploadlists/ and give it the created geneIDs.txt file, ask it to convert
# entrez gene to uniprot ID. Then download this as uniprotIDs.txt (as uncompressed, mapping table)

##### Import the list of uniprot IDs

In [10]:
# Import manually generated file of geneID --> uniprotID
df_uni_ids = pd.read_csv('uniprotIDs.txt', sep='\t')
df_uni_ids.columns = ['GeneID', 'UniprotID']
df_uni_ids['GeneID'] = df_uni_ids.GeneID.astype(str)

In [11]:
# Some of the GeneIds are actually a couple of IDs on one row. Split them into multiple rows
df_uni_ids = pd.concat([pd.Series(row['UniprotID'], row['GeneID'].split(','))              
                    for _, row in df_uni_ids.iterrows()]).reset_index()
df_uni_ids.columns = ['GeneID', 'UniprotID']

##### Create merged df to enable grouping by chemicalID and diseaseID

In [12]:
# Create mapping file of gene ID to chem ID... and gene ID to disease ID 
cg_dict = dict(zip(df_cg.GeneID, df_cg.ChemicalID))
cgd_dict = dict(zip(df_cgd.GeneID, df_cgd.DiseaseID))

In [13]:
df_uni_ids.head()

,GeneID,UniprotID
0,1,P04217
1,1,V9HWD8
2,2,P01023
3,131076,C9JQ41
4,131076,Q4VC31


In [14]:
df_uni_ids_d = df_uni_ids.copy() # make a copy for diseases

In [15]:
# Apply the mapping, thereby creating chemical column
df_uni_ids['ChemicalID'] = df_uni_ids.GeneID
df_uni_ids['ChemicalID'] = df_uni_ids['ChemicalID'].astype(int).map(cg_dict)

In [16]:
# Apply the mapping, thereby creating disease column
df_uni_ids_d['DiseaseID'] = df_uni_ids_d.GeneID
df_uni_ids_d['DiseaseID'] = df_uni_ids_d['DiseaseID'].astype(int).map(cgd_dict)

In [17]:
# # Output disease list, later used in nn notebook
# disease_list = df_uni_ids_d.DiseaseID.unique()
# len(disease_list)
# np.savetxt(r'diseases.lst', disease_list, fmt='%s')

In [18]:
## This establishes that the len of a disease id is always 8 while a chem is 7 or 10
# df_uni_ids['IDlen'] = df_uni_ids.ChemicalID.map(lambda x: len(x))
# df_uni_ids_d['IDlen'] = df_uni_ids_d.DiseaseID.map(lambda x: len(x))
# print(df_uni_ids.IDlen.unique())
# print(df_uni_ids_d.IDlen.unique())

##### Group by Chem ID

In [19]:
df_uni_ids.dropna().shape # 33381

(33381, 3)

In [20]:
df_uni_ids.head()

,GeneID,UniprotID,ChemicalID
0,1,P04217,D015032
1,1,V9HWD8,D015032
2,2,P01023,D015032
3,131076,C9JQ41,D014635
4,131076,Q4VC31,D014635


In [21]:
df_uni_ids_d.dropna().shape #2914

(14788, 3)

### 2. Mine goa file, attaching go function for each uniprot ID to the parent chemical/gene

In [22]:
# import goa file (uniprot ID to go_functions)
go_funcs = pd.read_csv('../goa_human.gaf', header=None, skiprows=30, sep='\t')

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
# Cut out all cols except uniprot ids and go_funcs, rename these
go_funcs = go_funcs.rename(columns={ go_funcs.columns[1]: "UniprotID" })
go_funcs = go_funcs.rename(columns={ go_funcs.columns[4]: "gofunc" })
col_list = ['UniprotID', 'gofunc']
df_go = go_funcs[col_list]

In [24]:
# Merge the go functions into our existing chem-uniprotID and dis-uniprotID dfs
df_uni_ids_d = df_uni_ids_d.merge(df_go, on='UniprotID', how='outer').dropna()
df_uni_ids = df_uni_ids.merge(df_go, on='UniprotID', how='outer').dropna()

### 3. Prep and write to output file 

In [25]:
df_uni_ids_d.head()

,GeneID,UniprotID,DiseaseID,gofunc
0,1,P04217,MESH:D012559,GO:0002576
1,1,P04217,MESH:D012559,GO:0003674
2,1,P04217,MESH:D012559,GO:0005576
3,1,P04217,MESH:D012559,GO:0005576
4,1,P04217,MESH:D012559,GO:0005576


In [75]:
df_uni_ids.head()

,GeneID,UniprotID,ChemicalID,gofunc,go_url
0,1,P04217,D015032,GO:0002576,<http://purl.obolibrary.org/obo/GO_0002576>
1,1,P04217,D015032,GO:0003674,<http://purl.obolibrary.org/obo/GO_0003674>
2,1,P04217,D015032,GO:0005576,<http://purl.obolibrary.org/obo/GO_0005576>
3,1,P04217,D015032,GO:0005576,<http://purl.obolibrary.org/obo/GO_0005576>
4,1,P04217,D015032,GO:0005576,<http://purl.obolibrary.org/obo/GO_0005576>


In [26]:
# Create a col with the full go url
df_uni_ids['go_url'] = '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids.gofunc.str.replace(':', '_')  + '>'
df_uni_ids_d['go_url'] =  '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids_d.gofunc.str.replace(':', '_')  + '>'

In [27]:
# Grab just the columns we want to output (diseaseID and go_url/ chemicalID and go_url)
col_list_c = ['ChemicalID', 'go_url']
col_list_d = ['DiseaseID', 'go_url']
df_c = df_uni_ids[col_list_c]
df_d = df_uni_ids_d[col_list_d]

In [28]:
# df_c.ChemicalID.unique()

In [29]:
df_d.DiseaseID.unique()

array(['MESH:D012559', 'MESH:D009404', 'MESH:D001749', ...,
       'MESH:C536489', 'MESH:C536531', 'MESH:C564161'], dtype=object)

In [30]:
# Output an association file for each of chem and dis
np.savetxt(r'associations_c.txt', df_c.values, fmt='%s')
np.savetxt(r'associations_d.txt', df_d.values, fmt='%s')

In [31]:
# Merge these two into one single file
subprocess.call('cat associations_* > myassociations', shell=True)

0

In [32]:
# Create entities.lst to inform opa2vec which entities we want vectors for
entities = df_d.DiseaseID.unique().tolist() + df_c.ChemicalID.unique().tolist()
np.savetxt(r'entities.lst', entities, fmt='%s')

In [33]:
## NEXT STEP in pipeline is to manually run opa2vec, though it could be automated here...

#### Export GoFunction counts per chem and per disease

In [92]:
# Create df of counts of chem and disease gofuncs for export, later used in assessing NN results
chem_gofunc_counts = df_uni_ids.groupby('ChemicalID')['gofunc'].nunique()
dis_gofunc_counts = df_uni_ids_d.groupby('DiseaseID')['gofunc'].nunique()

chem_gofunc_counts = pd.DataFrame(chem_gofunc_counts)
chem_gofunc_counts = chem_gofunc_counts.reset_index()
dis_gofunc_counts = pd.DataFrame(dis_gofunc_counts)
dis_gofunc_counts = dis_gofunc_counts.reset_index()

gofunc_counts = chem_gofunc_counts.merge(dis_gofunc_counts, how='outer')
gofunc_counts.to_csv('gofunc_counts.csv', index=False)